In [9]:
# --- AVU_schedule_only CELL 1 (clean): Parameters, Paths, Filters, Locks, History Bootstrap ---

import os, json
from datetime import datetime
from pathlib import Path

# ----------------------- 0) Load incoming parameters -----------------------
def _load_params():
    # Highest priority: server passes AVU_PARAMS (JSON string)
    raw = os.environ.get("AVU_PARAMS", "")
    if raw.strip():
        try:
            return json.loads(raw)
        except Exception:
            pass
    # Fallback: local file drop when debugging
    rp = Path.cwd() / "_run_params.json"
    if rp.exists():
        try:
            return json.loads(rp.read_text(encoding="utf-8"))
        except Exception:
            pass
    return {}

PARAMS = _load_params()

# Also allow Papermill parameters if executed that way
pm_params = {}
try:
    from papermill import get_parameters
    pm_params = get_parameters() or {}
except Exception:
    pm_params = {}

# ----------------------- 1) Year & Week (single sources of truth) -----------------------
def _coerce_week(x, default_week=None):
    if default_week is None:
        default_week = int(datetime.now().isocalendar().week)
    try:
        w = int(x)
    except Exception:
        w = int(default_week)
    if not (1 <= w <= 53):
        w = min(53, max(1, w))
    return w

def _coerce_year(x, default_year=None):
    if default_year is None:
        default_year = int(datetime.now().year)
    try:
        y = int(x)
    except Exception:
        y = int(default_year)
    return y

WEEK_NUMBER = _coerce_week(
    PARAMS.get("week_number")
    or pm_params.get("week_number")
    or os.getenv("WEEK_NUMBER")
    or datetime.now().isocalendar().week
)
week_number = WEEK_NUMBER  # legacy alias

CALENDAR_YEAR = _coerce_year(
    PARAMS.get("calendar_year")
    or pm_params.get("calendar_year")
    or os.getenv("CALENDAR_YEAR")
    or datetime.now().year
)
calendar_year = CALENDAR_YEAR  # legacy alias

# ----------------------- 2) Paths (single, consistent stack) -----------------------
# Input base
IRON_DATA = (
    PARAMS.get("input_path")
    or pm_params.get("input_path")
    or os.environ.get("INPUT_PATH")
    or r"C:\Users\Public\AVU\IRON_DATA"
)
BASE = Path(IRON_DATA)

# Output root (everything we write goes under here)
OUTPUT_PATH = Path(
    PARAMS.get("output_path")
    or pm_params.get("output_path")
    or os.environ.get("OUTPUT_PATH")
    or (Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA")
)
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

# Keep legacy alias some cells expect
IRON_DATA_PATH = OUTPUT_PATH

# Source files (catalogs, stock, etc.)
SOURCE_PATH = Path(PARAMS.get("source_path") or pm_params.get("source_path") or BASE)
SOURCE_PATH.mkdir(parents=True, exist_ok=True)

# Where we persist locks & history
LOCKED_PATH  = OUTPUT_PATH / "locked_weeks"; LOCKED_PATH.mkdir(parents=True, exist_ok=True)
HISTORY_DIR  = OUTPUT_PATH / "history";      HISTORY_DIR.mkdir(parents=True, exist_ok=True)
HIST_PATH    = HISTORY_DIR / "wine_campaign_history.json"

# ----------------------- 3) History map for cooldown/recency scoring -----------------------
def _hist_key(id_str, wine, vintage):
    sid = str(id_str or "").strip().replace(".0","")
    if sid: return sid
    return f"{str(wine or '').strip()}::{str(vintage or 'NV').strip()}"

def _load_history_map():
    candidates = [
        OUTPUT_PATH / "history" / "wine_campaign_history.json",
        IRON_DATA_PATH / "_output" / "history" / "wine_campaign_history.json",
        IRON_DATA_PATH / "history" / "wine_campaign_history.json",
    ]
    for p in candidates:
        if p.exists():
            try:
                m = json.loads(p.read_text(encoding="utf-8"))
                # normalize keys
                out = {}
                for k, v in m.items():
                    nk = _hist_key(k, v.get("wine"), v.get("vintage"))
                    out[nk] = v
                return out
            except Exception as e:
                print(f"⚠️ Bad history file {p.name}: {e}")
    return {}

HISTORY_MAP = _load_history_map()
print(f"🗂  History keys loaded: {len(HISTORY_MAP)}")

# ----------------------- 4) UI inputs (filters, selection, locks) -----------------------
NOTEBOOKS_PATH = Path("notebooks")
def _load_json_or_empty(p: Path):
    try:
        if p.exists() and p.stat().st_size > 0:
            return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        pass
    return {}

# Incoming from server (preferred)
UI_FILTERS_RAW      = PARAMS.get("filters")
LOCKED_SNAPSHOT_RAW = PARAMS.get("locked_calendar")
UI_SELECTION        = PARAMS.get("ui_selection") or None

# Local debug fallbacks
if UI_FILTERS_RAW is None:
    UI_FILTERS_RAW = _load_json_or_empty(NOTEBOOKS_PATH / "filters.json")
if LOCKED_SNAPSHOT_RAW is None:
    LOCKED_SNAPSHOT_RAW = _load_json_or_empty(NOTEBOOKS_PATH / "locked_calendar.json")

# Canonical days/slots
DAYS = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
NUM_SLOTS = 5

# If UI asked for a different year/week, override now (after we have UI_SELECTION)
if UI_SELECTION:
    WEEK_NUMBER  = _coerce_week(UI_SELECTION.get("week") or UI_SELECTION.get("week_number") or WEEK_NUMBER)
    week_number  = WEEK_NUMBER
    CALENDAR_YEAR = _coerce_year(UI_SELECTION.get("year") or CALENDAR_YEAR)
    calendar_year = CALENDAR_YEAR

print(f"📅 Year/Week selected: {CALENDAR_YEAR}-W{WEEK_NUMBER}")

# ----------------------- 5) Normalize filters -----------------------
def _price_bucket_from_01(v):
    try:
        x = max(0.0, min(1.0, float(v)))
    except Exception:
        return None
    if x < 0.25: return "Budget"
    if x < 0.50: return "Mid-range"
    if x < 0.75: return "Premium"
    return "Ultra Luxury"

def _canon_tier_name(s):
    t = str(s or "").strip().lower()
    if not t: return ""
    if "ultra" in t:   return "Ultra Luxury"
    if "luxury" in t:  return "Luxury"
    if "premium" in t: return "Premium"
    if "mid" in t:     return "Mid-range"
    if "budget" in t or "entry" in t: return "Budget"
    return ""

def _canon_tiers(seq):
    if not seq: return []
    out = []
    for x in seq:
        n = _canon_tier_name(x)
        if n and n not in out:
            out.append(n)
    return out

def _canon_day(s):
    s = str(s or "").strip()
    for d in DAYS:
        if s.lower() == d.lower():
            return d
    return None

def _to_int_or_none(v):
    try:
        return int(v)
    except Exception:
        return None

def normalize_filters(raw: dict) -> dict:
    raw = raw or {}
    price_bucket = raw.get("price_tier_bucket") or _price_bucket_from_01(raw.get("price_tier"))
    price_bucket = _canon_tier_name(price_bucket)

    price_tiers = _canon_tiers(raw.get("price_tiers") or [])
    if price_bucket and price_bucket not in price_tiers:
        price_tiers = [price_bucket] + [t for t in price_tiers if t != price_bucket]

    bottle_size = _to_int_or_none(raw.get("bottle_size"))

    loyalty = (raw.get("loyalty") or raw.get("loyalty_level") or "all").strip().lower()
    loyalty_levels = [str(x).strip().lower() for x in (raw.get("loyalty_levels") or []) if str(x).strip()]

    wine_type = raw.get("wine_type")
    if wine_type is not None and str(wine_type).strip().lower() == "all":
        wine_type = None

    # NEW: blocklists (ids or composite keys "Wine::Vintage")
    blocked_ids  = [str(x).strip() for x in (raw.get("blocked_ids")  or []) if str(x).strip()]
    blocked_keys = [str(x).strip() for x in (raw.get("blocked_keys") or []) if str(x).strip()]

    normalized = {
        "loyalty": loyalty,
        "loyalty_levels": loyalty_levels,
        "wine_type": wine_type,
        "bottle_size": bottle_size,
        "price_tier_bucket": price_bucket or "",
        "price_tiers": price_tiers,
        "last_stock": bool(raw.get("last_stock", False)),
        "last_stock_threshold": _to_int_or_none(raw.get("last_stock_threshold")) if raw.get("last_stock") else None,
        "seasonality_boost": bool(raw.get("seasonality_boost", False)),
        "style": (raw.get("style") or "default").strip().lower(),
        "calendar_day": _canon_day(raw.get("calendar_day")),
        "blocked_ids": blocked_ids,
        "blocked_keys": blocked_keys,
    }
    return normalized

UI_FILTERS = normalize_filters(UI_FILTERS_RAW)
HAS_UI_FILTERS = bool(UI_FILTERS)

# ----------------------- 6) Locks: persisted + UI overlay (slot-aware, year-aware) -----------------------
def _load_json_or_empty_quiet(p: Path):
    try:
        if p.exists() and p.stat().st_size > 0:
            return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        pass
    return {}

def _ensure_slots(arr, n=NUM_SLOTS):
    if arr is None:
        return [None]*n
    out = list(arr[:n])
    while len(out) < n:
        out.append(None)
    return out

def _merge_day_slots(base_day, overlay_day):
    base_arr = _ensure_slots(base_day, NUM_SLOTS)
    over_arr = _ensure_slots(overlay_day, NUM_SLOTS)
    merged = []
    for i in range(NUM_SLOTS):
        merged.append(over_arr[i] if over_arr[i] not in (None, "", "null") else base_arr[i])
    return merged

def get_effective_locks(year: int, week: int):
    yr_path = LOCKED_PATH / f"locked_calendar_{int(year)}_week_{int(week)}.json"
    wk_path = LOCKED_PATH / f"locked_calendar_week_{int(week)}.json"  # legacy
    persisted = _load_json_or_empty_quiet(yr_path) or _load_json_or_empty_quiet(wk_path)
    overlay   = LOCKED_SNAPSHOT_RAW if int(week) == int(WEEK_NUMBER) else {}
    if not overlay:
        src = "persisted only" if persisted else "none"
        print(f"🔒 Effective locks source: {src}")
        return persisted or {}
    out = dict(persisted or {})
    for day, over in (overlay or {}).items():
        day_norm = next((d for d in DAYS if str(day).lower() == d.lower()), None)
        if not day_norm:
            continue
        out[day_norm] = _merge_day_slots((persisted or {}).get(day_norm), over)
    print("🔒 Effective locks source: persisted + UI overlay (slot-aware)")
    return out

EFFECTIVE_LOCKS = get_effective_locks(CALENDAR_YEAR, WEEK_NUMBER)
LOCKED_CALENDAR = EFFECTIVE_LOCKS
HAS_UI_LOCKS = bool(LOCKED_SNAPSHOT_RAW)
HAS_PERSISTED_LOCKS = bool(LOCKED_CALENDAR)

# ----------------------- 7) Log summary -----------------------
print(f"✅ Source data path:   {SOURCE_PATH}")
print(f"✅ Output data path:   {OUTPUT_PATH}")
print(f"📦 Locked weeks path:  {LOCKED_PATH}")
print(f"📚 History map file:   {HIST_PATH.name} (exists: {HIST_PATH.exists()})")
print(f"📅 Using ISO Week:     {WEEK_NUMBER} (Year {CALENDAR_YEAR})")
print(f"🧩 Filters provided:   {'yes' if HAS_UI_FILTERS else 'no'}")
print(f"   → Normalized filters: {json.dumps(UI_FILTERS, indent=2)}")
print(f"🔒 UI locked snapshot: {'yes' if HAS_UI_LOCKS else 'no'}")
print(f"💾 Persisted locks:    {'yes' if HAS_PERSISTED_LOCKS else 'no'}")


🗂  History keys loaded: 53
📅 Year/Week selected: 2025-W34
🔒 Effective locks source: persisted only
✅ Source data path:   C:\Users\Public\AVU\IRON_DATA
✅ Output data path:   C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA
📦 Locked weeks path:  C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\locked_weeks
📚 History map file:   wine_campaign_history.json (exists: True)
📅 Using ISO Week:     34 (Year 2025)
🧩 Filters provided:   yes
   → Normalized filters: {
  "loyalty": "all",
  "loyalty_levels": [],
  "wine_type": null,
  "bottle_size": null,
  "price_tier_bucket": "",
  "price_tiers": [],
  "last_stock": false,
  "last_stock_threshold": null,
  "seasonality_boost": false,
  "style": "default",
  "calendar_day": null,
  "blocked_ids": [],
  "blocked_keys": []
}
🔒 UI locked snapshot: no
💾 Persisted locks:    yes


In [4]:
# ---  CELL 2: Attach last_campaign_date in schedule-only run ---
from pathlib import Path
import json

# Reuse normalized HISTORY_MAP from Cell 1 when available; else load & normalize
def _hist_key(id_str, wine, vintage):
    sid = str(id_str or "").strip().replace(".0", "")
    if sid:
        return sid
    return f"{str(wine or '').strip()}::{str(vintage or 'NV').strip()}"

HIST_PATH = OUTPUT_PATH / "history" / "wine_campaign_history.json"

if "HISTORY_MAP" in globals() and isinstance(HISTORY_MAP, dict) and HISTORY_MAP:
    _HIST = HISTORY_MAP
else:
    try:
        raw_hist = json.loads(HIST_PATH.read_text(encoding="utf-8")) if HIST_PATH.exists() else {}
    except Exception:
        raw_hist = {}
    _HIST = {}
    try:
        for k, v in (raw_hist or {}).items():
            _HIST[_hist_key(k, v.get("wine"), v.get("vintage"))] = v
    except Exception:
        # fallback: use as-is if normalization fails
        _HIST = raw_hist if isinstance(raw_hist, dict) else {}

def _last_campaign_for(rec: dict):
    rid  = str(rec.get("id", "")).strip().replace(".0", "")
    wine = (rec.get("wine") or rec.get("name") or "").strip()
    vint = str(rec.get("vintage") or "NV").strip()
    hit  = _HIST.get(rid) or _HIST.get(f"{wine}::{vint}") or {}
    return hit.get("last_campaign_date")

NUM_SLOTS = int(globals().get("NUM_SLOTS", 5))

if "weekly_calendar" in globals() and isinstance(weekly_calendar, dict):
    for d, items in weekly_calendar.items():
        out = []
        for it in (items or []):
            if not it:
                continue
            if not it.get("last_campaign_date"):
                lc = _last_campaign_for(it)
                if lc:
                    it["last_campaign_date"] = lc
            out.append(it)
        weekly_calendar[d] = out[:NUM_SLOTS]
    if "ui_output_data" in globals() and isinstance(ui_output_data, dict):
        ui_output_data["weekly_calendar"] = weekly_calendar
    print("🧠 schedule-only: attached last_campaign_date where available.")
else:
    print("ℹ️ weekly_calendar not present; skipping history attachment.")


ℹ️ weekly_calendar not present; skipping history attachment.


In [5]:
# --- CELL 3 — AVU_schedule_only: REGenerate Recommendations & Summaries (listens to UI filters) ---

from pathlib import Path
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

try:
    from IPython.display import display  # for previews if available
except Exception:
    pass

# tqdm text mode (avoid Jupyter widget errors)
try:
    from tqdm import tqdm
except Exception:
    def tqdm(x, **k): return x

# === Parameters ===
top_n = 3

# Prefer paths from Cell 1; fallback if not defined
try:
    OUTPUT_PATH = IRON_DATA_PATH
except NameError:
    OUTPUT_PATH = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA"

# === Load required files ===
client_pref_path = OUTPUT_PATH / "client_pref_df_latest.pkl"
cpi_path         = OUTPUT_PATH / "cpi_matrix_latest.pkl"

# stock can be named either way in different flows
stock_candidates = [
    OUTPUT_PATH / "stock_df_with_seasonality.pkl",
    OUTPUT_PATH / "stock_df_final.pkl",
]
stock_path = next((p for p in stock_candidates if p.exists()), None)

if not all([client_pref_path.exists(), cpi_path.exists(), stock_path is not None]):
    raise FileNotFoundError(
        f"❌ Required files missing. "
        f"client_pref_df_latest.pkl: {client_pref_path.exists()}, "
        f"cpi_matrix_latest.pkl: {cpi_path.exists()}, "
        f"stock_df_(with_seasonality|final).pkl: {stock_path is not None}"
    )

client_pref_df = pd.read_pickle(client_pref_path)
stock_df       = pd.read_pickle(stock_path).copy()
cpi_df         = pd.read_pickle(cpi_path).copy()

# === Helpers (filters) ===
def _canon_tier_name(s):
    t = str(s or "").strip().lower()
    if not t: return ""
    if "ultra" in t: return "Ultra Luxury"
    if "luxury" in t: return "Luxury"
    if "premium" in t: return "Premium"
    if "mid" in t: return "Mid-range"
    if "budget" in t or "entry" in t: return "Budget"
    return ""

def _canon_tiers(seq):
    if not seq: return []
    out = []
    for x in seq:
        n = _canon_tier_name(x)
        if n and n not in out:
            out.append(n)
    return out

def _week_to_season(week_no: int):
    if 1 <= week_no <= 8 or 49 <= week_no <= 53: return "Winter"
    if 9 <= week_no <= 22:  return "Spring"
    if 23 <= week_no <= 35: return "Summer"
    if 36 <= week_no <= 48: return "Autumn"
    return "Unknown"

try:
    _WEEK_NUMBER = int(week_number)  # from Cell 1
except Exception:
    _WEEK_NUMBER = int(datetime.now().isocalendar().week)

_SELECTED_SEASON = _week_to_season(_WEEK_NUMBER)

def _matches_wine_type_row(row, want):
    if not want: 
        return True
    want = str(want).strip().lower()
    full_type = str(row.get('full_type', '')).lower()
    typ       = str(row.get('type', '')).lower()
    color     = str(row.get('color', '')).lower()

    # common intents: red / white / rosé / sparkling / sweet
    if want in full_type or want in typ:
        return True
    if want in ("rose", "rosé") and ("rosé" in full_type or "rose" in full_type or "rosé" in typ or "rose" in typ):
        return True
    if want == "red" and "red" in color:
        return True
    if want == "white" and "white" in color:
        return True
    if want.startswith("spark") and ("sparkling" in full_type or "sparkling" in typ):
        return True
    if want.startswith("sweet") and ("sweet" in full_type or "sweet" in typ):
        return True
    return False

def _apply_ui_filters(stock_df, UI_FILTERS):
    df = stock_df.copy()

    # Ensure common columns exist / normalized
    df.columns = df.columns.str.strip()
    if 'price_tier' in df.columns:
        df['price_tier'] = df['price_tier'].map(_canon_tier_name)
    else:
        df['price_tier'] = ""

    if 'stock' not in df.columns:
        df['stock'] = 0
    df['stock'] = pd.to_numeric(df['stock'], errors='coerce').fillna(0).astype(int)

    # full_type if missing
    if 'full_type' not in df.columns:
        tcol = 'type' if 'type' in df.columns else None
        ccol = 'color' if 'color' in df.columns else None
        if tcol and ccol:
            df['full_type'] = df[tcol].astype(str).str.title().str.strip() + " " + df[ccol].astype(str).str.title().str.strip()
        elif tcol:
            df['full_type'] = df[tcol].astype(str).str.title().str.strip()
        else:
            df['full_type'] = "Unknown"

    # bottle size (prefer ml)
    if 'bottle_size_ml' not in df.columns:
        # try to parse from 'size' or 'size_cl'
        def _to_ml(x):
            try:
                s = str(x).lower().strip().replace('ml','').replace('cl','')
                v = float(s)
                return int(round(v*10)) if v < 100 else int(round(v))
            except:
                return np.nan
        if 'size' in df.columns:
            df['bottle_size_ml'] = df['size'].apply(_to_ml)
        elif 'size_cl' in df.columns:
            df['bottle_size_ml'] = df['size_cl'].apply(_to_ml)
        else:
            df['bottle_size_ml'] = np.nan

    # ---- Apply filters (hard) ----
    # price tiers
    tiers = _canon_tiers(UI_FILTERS.get("price_tiers", []))
    single_bucket = _canon_tier_name(UI_FILTERS.get("price_tier_bucket", ""))
    if single_bucket and single_bucket not in tiers:
        tiers = [single_bucket] + [t for t in tiers if t != single_bucket]
    if tiers:
        before = len(df)
        df = df[df['price_tier'].isin(tiers)]
        print(f"🎛️ price_tiers {tiers} → kept {len(df)}/{before}")

    # wine type
    wt = UI_FILTERS.get("wine_type")
    if wt:
        before = len(df)
        df = df[df.apply(lambda r: _matches_wine_type_row(r, wt), axis=1)]
        print(f"🎛️ wine_type '{wt}' → kept {len(df)}/{before}")

    # bottle size: treat UI value as a minimum (so 'bigger' works naturally)
    bs = UI_FILTERS.get("bottle_size")
    if bs:
        try:
            bs = int(bs)
            before = len(df)
            df = df[df['bottle_size_ml'].fillna(0) >= bs]
            print(f"🎛️ bottle_size ≥ {bs} ml → kept {len(df)}/{before}")
        except Exception:
            pass

    # last stock
    if UI_FILTERS.get("last_stock"):
        raw_thr = UI_FILTERS.get("last_stock_threshold", None)
        try:
            thr = int(raw_thr) if raw_thr is not None and str(raw_thr).strip() != "" else 10
        except Exception:
            thr = 10
        before = len(df)
        df = df[df['stock'] <= thr]
        print(f"🎛️ last_stock ≤ {thr} → kept {len(df)}/{before}")

    # seasonality: prefer explicit season tag, else fallback to “same week ±7d last year” using last offer date
    if UI_FILTERS.get("seasonality_boost", False):
        before = len(df)
        if 'seasonality_boost' in df.columns:
            df = df[df['seasonality_boost'].apply(lambda x: _SELECTED_SEASON in x if isinstance(x, list) else False)]
            print(f"🎛️ seasonality={_SELECTED_SEASON} (tag) → kept {len(df)}/{before}")
        else:
            # fallback via OMT last offer date proximity
            date_col = None
            for cand in ['OMT last offer date', 'most_recent_date', 'Schedule DateTime']:
                if cand in df.columns:
                    date_col = cand; break
            if date_col:
                df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
                last_year = datetime.now() - timedelta(days=365)
                window_end = last_year + timedelta(days=7)
                df = df[df[date_col].between(last_year, window_end)]
            print(f"🎛️ seasonality={_SELECTED_SEASON} (date fallback) → kept {len(df)}/{before}")

    # Always require some minimum availability for scheduling
    before = len(df); 
    df = df[df['stock'] > 0]
    print(f"📦 enforce in-stock (>0) → kept {len(df)}/{before}")

    return df

def _relax_filters_if_empty(df_initial, UI_FILTERS):
    """If filters remove everything, progressively relax to avoid empty recs."""
    df = df_initial
    if not df.empty:
        return df, "strict"
    # 1) drop last_stock
    relaxed = dict(UI_FILTERS)
    if relaxed.get("last_stock"):
        relaxed["last_stock"] = False
        df = _apply_ui_filters(stock_df, relaxed)
        if not df.empty: return df, "no-last-stock"

    # 2) widen bottle size
    if relaxed.get("bottle_size"):
        relaxed["bottle_size"] = None
        df = _apply_ui_filters(stock_df, relaxed)
        if not df.empty: return df, "no-bottle-size"

    # 3) drop price tiers
    if relaxed.get("price_tiers") or relaxed.get("price_tier_bucket"):
        relaxed["price_tiers"] = []
        relaxed["price_tier_bucket"] = ""
        df = _apply_ui_filters(stock_df, relaxed)
        if not df.empty: return df, "no-tiers"

    # 4) drop wine type
    if relaxed.get("wine_type"):
        relaxed["wine_type"] = None
        df = _apply_ui_filters(stock_df, relaxed)
        if not df.empty: return df, "no-type"

    # 5) drop seasonality
    if relaxed.get("seasonality_boost"):
        relaxed["seasonality_boost"] = False
        df = _apply_ui_filters(stock_df, relaxed)
        if not df.empty: return df, "no-seasonality"

    # 6) fallback: all in-stock
    df = stock_df.copy()
    df['stock'] = pd.to_numeric(df.get('stock', 0), errors='coerce').fillna(0)
    df = df[df['stock'] > 0]
    return df, "fallback-instock"

# === ID normalization (keep as strings; drop trailing '.0' only) ===
def normalize_id_series(s):
    out = s.astype(str).str.strip()
    out = out.str.replace(r'\.0$', '', regex=True)  # handles float-like ids
    return out

# Normalize IDs
if 'id' in stock_df.columns:
    stock_df['id'] = normalize_id_series(stock_df['id'])
if 'id' in cpi_df.columns:
    cpi_df['id'] = normalize_id_series(cpi_df['id'])

# === Blocked items (IDs or "wine::vintage" keys) — from Cell 1 normalize_filters ===
blocked_ids  = set(str(x).strip() for x in (UI_FILTERS.get("blocked_ids")  or []) if str(x).strip())
blocked_keys = set(str(x).strip() for x in (UI_FILTERS.get("blocked_keys") or []) if str(x).strip())

if blocked_ids or blocked_keys:
    # Build keys on stock_df to drop both kinds
    def _make_key(row):
        rid = str(row.get("id", "")).strip()
        if rid: 
            return rid
        w = str(row.get("wine") or row.get("name") or "").strip()
        v = str(row.get("vintage") or "NV").strip()
        return f"{w}::{v}"

    stock_df["_blk_key"] = stock_df.apply(_make_key, axis=1)
    before = len(stock_df)
    stock_df = stock_df[~(stock_df["id"].astype(str).isin(blocked_ids) | stock_df["_blk_key"].isin(blocked_keys))].copy()
    stock_df.drop(columns=["_blk_key"], errors="ignore", inplace=True)
    after = len(stock_df)
    print(f"⛔ Blocked filter → removed {before - after} rows")

    # Keep CPI aligned (drop blocked ids from cpi_df)
    if "id" in cpi_df.columns and blocked_ids:
        cpi_before = len(cpi_df)
        cpi_df = cpi_df[~cpi_df["id"].astype(str).isin(blocked_ids)].copy()
        print(f"⛔ Blocked in CPI → removed {cpi_before - len(cpi_df)} rows")

# === Apply UI filters to STOCK and keep CPI aligned ===
try:
    UI_FILTERS  # from Cell 1
except NameError:
    UI_FILTERS = {}

filtered_stock_df = _apply_ui_filters(stock_df, UI_FILTERS)
filtered_stock_df, relax_reason = _relax_filters_if_empty(filtered_stock_df, UI_FILTERS)
print(f"🧭 filter strategy used: {relax_reason} (final candidates: {len(filtered_stock_df)})")

# Rebuild in-stock id set after filters
filtered_stock_df['stock'] = pd.to_numeric(filtered_stock_df['stock'], errors='coerce').fillna(0)
in_stock_ids = set(filtered_stock_df.loc[filtered_stock_df['stock'] > 0, 'id'])
cpi_df = cpi_df[cpi_df['id'].isin(in_stock_ids)].copy()

# === Final ID overlap confirmation ===
overlap_ids = set(filtered_stock_df['id']) & set(cpi_df['id'])
print(f"✅ Overlapping IDs after cleaning/filters: {len(overlap_ids)}")
print("🔍 Sample overlapping IDs:", list(overlap_ids)[:10])

# === Full Type + Metadata enrichment (guard missing columns) ===
filtered_stock_df.columns = filtered_stock_df.columns.str.strip()
if 'full_type' not in filtered_stock_df.columns:
    tcol  = 'type'  if 'type'  in filtered_stock_df.columns else None
    ccol  = 'color' if 'color' in filtered_stock_df.columns else None
    if tcol and ccol:
        filtered_stock_df['full_type'] = (
            filtered_stock_df[tcol].astype(str).str.title().str.strip() + " " +
            filtered_stock_df[ccol].astype(str).str.title().str.strip()
        )
    elif tcol:
        filtered_stock_df['full_type'] = filtered_stock_df[tcol].astype(str).str.title().str.strip()
    else:
        filtered_stock_df['full_type'] = "Unknown"

# Ensure campaign columns exist
for col in ['Num_of_CM', 'number_of_sent_emails', 'most_recent_date']:
    if col not in filtered_stock_df.columns:
        filtered_stock_df[col] = 0 if col != 'most_recent_date' else pd.NaT
filtered_stock_df['Num_of_CM'] = pd.to_numeric(filtered_stock_df['Num_of_CM'], errors='coerce').fillna(0).astype(int)
filtered_stock_df['number_of_sent_emails'] = pd.to_numeric(filtered_stock_df['number_of_sent_emails'], errors='coerce').fillna(0).astype(int)

# === Merge CPI with stock info (add full_type for grouping) ===
merged_cpi_df = cpi_df.merge(filtered_stock_df[['id', 'full_type']].copy(), on='id', how='left')
print("📦 Unique wine types:", merged_cpi_df['full_type'].dropna().unique())

# === Generate Recommendations (per client, top-N) ===
recommendations_list = []
value_vars = [col for col in merged_cpi_df.columns if col.startswith("pref_cpi_for_")]
merged_cpi_df = merged_cpi_df.loc[:, ~merged_cpi_df.columns.duplicated()].copy()

if not value_vars:
    print("⚠️ No CPI preference columns found (pref_cpi_for_*) — skipping recommendation build.")
    recommendations_df = pd.DataFrame()
else:
    for col in tqdm(value_vars, desc="🔄 Generating recommendations per client"):
        cid = col.replace("pref_cpi_for_", "")
        try:
            client_cpi_df = merged_cpi_df[['id', 'full_type', col]].copy()
            client_cpi_df.columns = ['id', 'full_type', 'cpi_score']
            client_cpi_df = client_cpi_df[client_cpi_df['id'].isin(in_stock_ids)]

            top_wines = (
                client_cpi_df.sort_values(by='cpi_score', ascending=False)
                .head(top_n)
                .assign(customer_no=cid)
            )
            recommendations_list.append(top_wines)
        except KeyError:
            print(f"⚠️ Skipping {col} — missing or malformed.")

    recommendations_df = pd.concat(recommendations_list, ignore_index=True) if recommendations_list else pd.DataFrame()

# === Load (optional) summary created by the engine (used just for printing) ===
top3_by_type_path = OUTPUT_PATH / "top3_recommendations_per_client_by_type.pkl"
top3_by_type = pd.read_pickle(top3_by_type_path) if top3_by_type_path.exists() else pd.DataFrame()

# === PART 2: If no recs, short-circuit and prep placeholders ===
if recommendations_df.empty:
    print("\n⚠️ No recommendations found with current filters. UI will rely on fallback selection.")
    locked_df = pd.DataFrame()  # will be built in Cell 5 onward
else:
    # === Enrich recommendations_df with stock metadata & context ===
    enriched_df = recommendations_df.merge(
        filtered_stock_df[['id', 'wine', 'vintage', 'grape_list', 'stock', 'full_type']].copy(),
        on='id', how='left'
    )

    # Fallback columns (prevent errors in grouping)
    for col in ['full_type', 'wine', 'vintage', 'grape_list', 'stock']:
        if col not in enriched_df.columns:
            print(f"⚠️ Column '{col}' missing after merge — injecting fallback.")
            enriched_df[col] = 'Unknown' if col != 'stock' else 0

    locked_df = pd.DataFrame()  # placeholder for later cells

    # === Small previews / diagnostics ===
    print(f"Total unique clients in recommendations: {enriched_df['customer_no'].nunique()}")
    if not top3_by_type.empty and 'full_type' in top3_by_type.columns:
        print("\n🍷 Engine Top 3 per wine type (preview):")
        for wine_type in top3_by_type['full_type'].dropna().unique():
            sub = top3_by_type[top3_by_type['full_type'] == wine_type][
                ['wine', 'vintage', 'avg_cpi_score', 'times_recommended', 'grape_list', 'stock']
            ]
            try:
                display(sub.round(3).head(3))
            except Exception:
                print(sub.round(3).head(3).to_string(index=False))

# Final diagnostics
print("🧪 Number of CPI columns:", len(value_vars))
print("🧪 First few CPI columns:", value_vars[:5])
print("🔍 Sample overlapping IDs:", list(overlap_ids)[:10] if overlap_ids else "∅")


📦 enforce in-stock (>0) → kept 4475/4498
🧭 filter strategy used: strict (final candidates: 4475)
✅ Overlapping IDs after cleaning/filters: 2299
🔍 Sample overlapping IDs: ['48958', '63040', '56799', '51516', '63490', '35774', '13024', '62185', '11560', '53976']
📦 Unique wine types: ['Still Red' 'Still White' 'Sweet White' 'Sparkling Unknown'
 'Sparkling White' 'Sparkling Rose' 'Still Rose' 'Unknown Red' 'Sweet Red'
 'Sweet Unknown' 'Fortified White']


🔄 Generating recommendations per client: 100%|██████████| 1951/1951 [00:02<00:00, 917.33it/s]


⚠️ Column 'full_type' missing after merge — injecting fallback.
Total unique clients in recommendations: 67

🍷 Engine Top 3 per wine type (preview):


,wine,vintage,avg_cpi_score,times_recommended,grape_list,stock
8,Champagne Brut Impérial Rosé,NV,0.963,1,Chardonnay/Pinot Noir,12
32,Champagne Extra Brut R.D.,2007,0.963,1,Chardonnay/Pinot Noir,6
49,Champagne Brut Blanc de Blancs,2014,0.963,1,Chardonnay/Pinot Noir,6


🧪 Number of CPI columns: 1951
🧪 First few CPI columns: ['pref_cpi_for_101332', 'pref_cpi_for_101332', 'pref_cpi_for_101332', 'pref_cpi_for_101332', 'pref_cpi_for_101332']
🔍 Sample overlapping IDs: ['48958', '63040', '56799', '51516', '63490', '35774', '13024', '62185', '11560', '53976']


In [6]:
# --- CELL 4 — AVU_schedule_only: UI cell interactions and commands (normalized) ---
import json
import os
from pathlib import Path

FILTERS_PATH = Path("notebooks") / "filters.json"

# ---------- Helpers (used only if Cell 1's normalize_filters isn't in scope) ----------
def _canon_tier_name(s):
    t = str(s or "").strip().lower()
    if not t: return ""
    if "ultra" in t: return "Ultra Luxury"
    if "luxury" in t: return "Luxury"
    if "premium" in t: return "Premium"
    if "mid" in t: return "Mid-range"
    if "budget" in t or "entry" in t: return "Budget"
    return ""

def _canon_tiers(seq):
    out = []
    for x in (seq or []):
        n = _canon_tier_name(x)
        if n and n not in out:
            out.append(n)
    return out

def _canon_day(s, DAYS=("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")):
    s = str(s or "").strip()
    for d in DAYS:
        if s.lower() == d.lower():
            return d
    return None

def _to_ml(x):
    """
    Normalize numeric size to milliliters.
    Accepts values that may be in ml (>= 100) or cl (< 100).
    Strings like '1500', '150cl', '75cl' are handled.
    """
    if x is None or x == "":
        return None
    s = str(x).strip().lower().replace("ml","").replace("cl","")
    try:
        v = float(s)
    except Exception:
        return None
    # <100 → treat as cl → *10 ; else already ml
    return int(round(v*10)) if v < 100 else int(round(v))

def _price_bucket_from_01(v):
    try:
        x = max(0.0, min(1.0, float(v)))
    except Exception:
        return None
    if x < 0.25: return "Budget"
    if x < 0.50: return "Mid-range"
    if x < 0.75: return "Premium"
    return "Ultra Luxury"

def _normalize_filters_local(raw: dict) -> dict:
    raw = raw or {}
    price_bucket = raw.get("price_tier_bucket") or _price_bucket_from_01(raw.get("price_tier"))
    price_bucket = _canon_tier_name(price_bucket)

    price_tiers = _canon_tiers(raw.get("price_tiers") or [])
    if price_bucket and price_bucket not in price_tiers:
        price_tiers = [price_bucket] + [t for t in price_tiers if t != price_bucket]

    last_stock = bool(raw.get("last_stock", False))
    lst = raw.get("last_stock_threshold")
    last_stock_threshold = int(lst) if (last_stock and str(lst or "").strip()) else None

    bottle_size = _to_ml(raw.get("bottle_size"))

    loyalty = (raw.get("loyalty") or raw.get("loyalty_level") or "all").strip().lower()
    loyalty_levels = [str(x).strip().lower() for x in (raw.get("loyalty_levels") or []) if str(x).strip()]

    wine_type = raw.get("wine_type")
    if wine_type is not None and str(wine_type).strip().lower() == "all":
        wine_type = None

    blocked_ids  = [str(x).strip() for x in (raw.get("blocked_ids")  or []) if str(x).strip()]
    blocked_keys = [str(x).strip() for x in (raw.get("blocked_keys") or []) if str(x).strip()]

    return {
        "loyalty": loyalty,
        "loyalty_levels": loyalty_levels,
        "wine_type": wine_type,
        "bottle_size": bottle_size,
        "price_tier_bucket": price_bucket or "",
        "price_tiers": price_tiers,
        "last_stock": last_stock,
        "last_stock_threshold": last_stock_threshold,
        "seasonality_boost": bool(raw.get("seasonality_boost", False)),
        "style": (raw.get("style") or "default").strip().lower(),
        "calendar_day": _canon_day(raw.get("calendar_day")),
        # NEW: blocks
        "blocked_ids": blocked_ids,
        "blocked_keys": blocked_keys,
    }

# ---------- 1) Load raw filters (non-fatal if missing/empty) ----------
raw_filters = {}
if FILTERS_PATH.exists() and FILTERS_PATH.stat().st_size > 0:
    try:
        raw_filters = json.loads(FILTERS_PATH.read_text(encoding="utf-8"))
    except Exception as e:
        print(f"⚠️ Could not parse {FILTERS_PATH.name}: {e}")

# ---------- 2) Build canonical UI_FILTERS used by downstream cells ----------
if 'normalize_filters' in globals() and callable(normalize_filters):
    # Trust Cell 1’s canonicalizer to avoid drift
    UI_FILTERS = normalize_filters(raw_filters)
else:
    UI_FILTERS = _normalize_filters_local(raw_filters)

# ---------- 3) Pretty print summary ----------
print("🎚 UI Filters (normalized):")
print(f"  • Last stock: {UI_FILTERS['last_stock']}"
      + (f" (≤ {UI_FILTERS['last_stock_threshold']})" if UI_FILTERS.get('last_stock') else ""))
print(f"  • Seasonality boost: {UI_FILTERS['seasonality_boost']}")
print(f"  • Wine type: {UI_FILTERS['wine_type'] or '—'}")
print(f"  • Bottle size (ml): {UI_FILTERS['bottle_size'] or '—'}")
tiers_line = UI_FILTERS['price_tiers'] or ('[' + UI_FILTERS['price_tier_bucket'] + ']' if UI_FILTERS['price_tier_bucket'] else '—')
print(f"  • Price tiers: {tiers_line}")
print(f"  • Loyalty: {UI_FILTERS['loyalty']}")
print(f"  • Style: {UI_FILTERS['style']}")
if UI_FILTERS.get('calendar_day'):
    print(f"  • Calendar day context: {UI_FILTERS['calendar_day']}")
# NEW: blocked preview
b_ids  = UI_FILTERS.get("blocked_ids") or []
b_keys = UI_FILTERS.get("blocked_keys") or []
if b_ids or b_keys:
    print(f"  • Blocked: {len(b_ids)} ids, {len(b_keys)} keys")

# 4) Expose UI_FILTERS to later cells (no further action required)


🎚 UI Filters (normalized):
  • Last stock: False
  • Seasonality boost: False
  • Wine type: —
  • Bottle size (ml): —
  • Price tiers: —
  • Loyalty: all
  • Style: default


In [7]:
# --- CELL 5 — AVU_schedule_only: Filters, helpers & locked normalization (robust) ---

from datetime import datetime, timedelta
from pathlib import Path
import json
import pandas as pd
import numpy as np

# Expect from previous cells:
# - week_number, IRON_DATA_PATH, stock_df, client_pref_df, top3_by_type (optional)
# - NUM_SLOTS, DAYS (from Cell 1) and EFFECTIVE_LOCKS (persisted + UI overlay)

# Week banner (safe)
try:
    _wk_print = int(week_number)
except Exception:
    _wk_print = int(datetime.now().isocalendar().week)
print(f"📅 Week selected: {_wk_print}")

# -----------------------------
# 1) Use normalized UI filters
# -----------------------------
try:
    _UF = UI_FILTERS  # normalized in Cell 1/4
except NameError:
    _UF = {}

print("🔍 Active UI filters (normalized):")
print(json.dumps(_UF, indent=2))

# -----------------------------
# 2) Safe stock/client prep
# -----------------------------
stock_df = stock_df.copy()
client_pref_df = client_pref_df.copy()

# Ensure essential columns exist
for col in ["id","wine","vintage","full_type","region_group","price_tier","stock"]:
    if col not in stock_df.columns:
        stock_df[col] = np.nan

# If wine names are blank but 'name' exists, use it
if stock_df["wine"].fillna("").eq("").all() and "name" in stock_df.columns:
    stock_df["wine"] = stock_df["name"]

# id as string (no trailing .0), stock numeric
stock_df["id"] = stock_df["id"].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
stock_df["stock"] = pd.to_numeric(stock_df["stock"], errors="coerce").fillna(0).astype(int)

# full_type robust build (if missing or empty)
if ("full_type" not in stock_df.columns) or stock_df["full_type"].fillna("").eq("").all():
    tcol = "type"  if "type"  in stock_df.columns else None
    ccol = "color" if "color" in stock_df.columns else None
    if tcol and ccol:
        stock_df["full_type"] = (
            stock_df[tcol].astype(str).str.title().str.strip() + " " +
            stock_df[ccol].astype(str).str.title().str.strip()
        )
    elif tcol:
        stock_df["full_type"] = stock_df[tcol].astype(str).str.title().str.strip()
    else:
        stock_df["full_type"] = "Unknown"

# region_group fallback
if "region_group" not in stock_df.columns:
    stock_df["region_group"] = stock_df.get("region", np.nan)
stock_df["region_group"] = stock_df["region_group"].fillna(stock_df.get("origin")).fillna("Unknown")

# Ensure 'occasion' exists (align with engine logic)
if "occasion" not in stock_df.columns:
    def _infer_occasion(price_tier, wine_type):
        pt = str(price_tier or "").title()
        wt = str(wine_type or "").lower()
        if pt in ("Luxury","Ultra Luxury"): return "Gifting"
        if "sparkling" in wt or "rosé" in wt or "rose" in wt: return "Celebration"
        if pt in ("Mid-range","Premium"): return "Dinner"
        return "Casual"
    stock_df["occasion"] = stock_df.apply(lambda r: _infer_occasion(r["price_tier"], r["full_type"]), axis=1)
else:
    stock_df["occasion"] = stock_df["occasion"].fillna("Casual").astype(str)

# Bottle size in ml (prefer existing column)
if "bottle_size_ml" not in stock_df.columns:
    def _to_ml(x):
        if x is None or (isinstance(x, float) and np.isnan(x)): return np.nan
        s = str(x).strip().lower().replace(" ", "")
        s = s.replace("ml","").replace("cl","")
        try:
            v = float(s)
        except Exception:
            return np.nan
        return v*10 if v < 100 else v  # 75cl -> 750
    if "size_cl" in stock_df.columns:
        stock_df["bottle_size_ml"] = pd.to_numeric(stock_df["size_cl"].apply(_to_ml), errors="coerce")
    elif "size" in stock_df.columns:
        stock_df["bottle_size_ml"] = pd.to_numeric(stock_df["size"].apply(_to_ml), errors="coerce")
    else:
        stock_df["bottle_size_ml"] = np.nan

# Normalize price_tier labels
def _canon_tier_name(s):
    t = str(s or "").strip().lower()
    if "ultra" in t: return "Ultra Luxury"
    if "luxury" in t: return "Luxury"
    if "premium" in t: return "Premium"
    if "mid" in t: return "Mid-range"
    if "budget" in t or "entry" in t: return "Budget"
    return ""
stock_df["price_tier"] = stock_df["price_tier"].apply(_canon_tier_name)

# -----------------------------
# 3) Apply UI filters to pools
# -----------------------------
# Price tiers (support array and single bucket)
pt_list = list(_UF.get("price_tiers") or [])
pt_bucket = _UF.get("price_tier_bucket") or ""
if pt_bucket and pt_bucket not in pt_list:
    pt_list.append(pt_bucket)
if pt_list:
    stock_df = stock_df[stock_df["price_tier"].isin(pt_list)].copy()

# Loyalty (client filter)
ll = _UF.get("loyalty_levels") or []
if ll and "loyalty_level" in client_pref_df.columns:
    client_pref_df = client_pref_df[
        client_pref_df["loyalty_level"].astype(str).str.lower().isin([x.lower() for x in ll])
    ].copy()

# Wine type from UI may be "Red"/"White"/"Sparkling" etc.
sel_type = _UF.get("wine_type")
if sel_type:
    mask = stock_df["full_type"].fillna("").str.contains(str(sel_type), case=False, na=False)
    stock_df = stock_df[mask].copy()

# Bottle size (ml)
sel_size_ml = _UF.get("bottle_size")
if sel_size_ml is not None:
    stock_df = stock_df[pd.to_numeric(stock_df["bottle_size_ml"], errors="coerce") == int(sel_size_ml)].copy()

# Seasonality boost (last 12 months window if column exists)
if _UF.get("seasonality_boost", False) and "OMT last offer date" in stock_df.columns:
    stock_df["OMT last offer date"] = pd.to_datetime(stock_df["OMT last offer date"], errors="coerce")
    cut_start = datetime.today() - timedelta(days=365)
    cut_end = cut_start + timedelta(days=7)
    stock_df = stock_df[stock_df["OMT last offer date"].between(cut_start, cut_end, inclusive="both")].copy()

# Last-stock filter (≤ threshold), min availability enforced later
if _UF.get("last_stock", False):
    thr = int(_UF.get("last_stock_threshold", 10))
    stock_df = stock_df[stock_df["stock"] <= thr].copy()

# -----------------------------
# 3b) Apply blocks (NEW)
# -----------------------------
_blocked_ids  = {str(x).strip() for x in (_UF.get("blocked_ids")  or []) if str(x).strip()}
_blocked_keys = {str(x).strip() for x in (_UF.get("blocked_keys") or []) if str(x).strip()}

def _mk_key_from_row(row) -> str:
    rid = str(row.get("id") or "").strip()
    if rid:
        return rid
    nm = str(row.get("wine") or "").strip()
    vt = str(row.get("vintage") or "NV").strip()
    return f"{nm}::{vt}"

if _blocked_ids or _blocked_keys:
    before = len(stock_df)
    stock_df = stock_df[
        (~stock_df["id"].astype(str).isin(_blocked_ids)) &
        (~stock_df.apply(_mk_key_from_row, axis=1).isin(_blocked_keys))
    ].copy()
    print(f"⛔ blocks applied → kept {len(stock_df)}/{before}")

# -----------------------------
# 4) Lock handling (from Cell 1)
# -----------------------------
try:
    locks_raw = EFFECTIVE_LOCKS or {}
except NameError:
    LOCKED_PATH = IRON_DATA_PATH / "locked_weeks"
    lock_file = LOCKED_PATH / f"locked_calendar_week_{_wk_print}.json"
    if lock_file.exists():
        try:
            locks_raw = json.loads(lock_file.read_text(encoding="utf-8"))
        except Exception:
            locks_raw = {}
    else:
        locks_raw = {}

NUM_SLOTS = int(NUM_SLOTS) if "NUM_SLOTS" in globals() else 5
DAYS = list(DAYS) if "DAYS" in globals() else ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

def _ensure_slots(arr, n=NUM_SLOTS):
    if arr is None: return [None]*n
    out = list(arr[:n])
    while len(out) < n: out.append(None)
    return out

# Flatten locked → DataFrame with slot indices
locked_rows = []
for day in DAYS:
    items = locks_raw.get(day) or []
    for idx, it in enumerate(_ensure_slots(items, NUM_SLOTS)):
        if not it: continue
        d = {
            "day": day,
            "slot": int(it.get("slot", idx)),
            "id": str(it.get("id") or "").strip(),
            "wine": str(it.get("wine") or it.get("name") or "").strip(),
            "vintage": str(it.get("vintage") or "").strip(),
            "locked": True
        }
        locked_rows.append(d)

locked_df = pd.DataFrame(locked_rows)
print(f"🔒 Locked slots in effect: {len(locked_df)}")

# -----------------------------
# 5) Weekly selection engine
# -----------------------------
DAY_TIERS = {
    "Monday":    ["Budget","Premium"],
    "Tuesday":   ["Premium"],
    "Wednesday": ["Luxury"],
    "Thursday":  ["Premium","Luxury"],
    "Friday":    ["Ultra Luxury"],
    "Saturday":  ["Luxury","Ultra Luxury"],
    "Sunday":    ["Budget","Luxury"],
}
DAY_OCCASION = {
    "Monday": "Casual", "Tuesday": "Casual", "Wednesday": "Dinner",
    "Thursday": "Dinner", "Friday": "Party", "Saturday": "Gifting", "Sunday": "Dinner"
}

def _norm(s):
    return str(s or "").strip().casefold()

# Segment frequency map from client preferences
if not client_pref_df.empty:
    rg_series = client_pref_df.get("region_group")
    ft_series = client_pref_df.get("full_type")
    rg_series = rg_series.astype(str).fillna("").map(_norm) if rg_series is not None else pd.Series([""]*len(client_pref_df))
    ft_series = ft_series.astype(str).fillna("").map(_norm) if ft_series is not None else pd.Series([""]*len(client_pref_df))

    seg_counts = (
        pd.DataFrame({"rg": rg_series, "ft": ft_series})
        .value_counts()
        .to_dict()
    )
else:
    seg_counts = {}

def _segment_score(row):
    rg = _norm(row.get("region_group"))
    ft = _norm(row.get("full_type"))
    return int(seg_counts.get((rg, ft), 0))

pool = stock_df.copy()
pool["segment_score"] = 0 if not seg_counts else pool.apply(_segment_score, axis=1)

# Minimum stock safeguard:
min_stock = 6 if not _UF.get("last_stock", False) else 3
pool = pool[pool["stock"] >= min_stock].copy()

# Used IDs + Keys (start with locked ones to avoid duplicates)
def _mk_key(it: dict) -> str:
    rid = str(it.get("id") or "").strip()
    if rid: return rid
    nm = str(it.get("wine") or it.get("name") or "").strip()
    vt = str(it.get("vintage") or "NV").strip()
    return f"{nm}::{vt}"

used_ids  = set(locked_df["id"].astype(str)) if not locked_df.empty else set()
used_keys = set()
if not locked_df.empty:
    for _, r in locked_df.iterrows():
        used_keys.add(_mk_key({"id": r.get("id"), "wine": r.get("wine"), "vintage": r.get("vintage")}))

# Respect blocks in the candidate pool too (key-level)
if _blocked_ids or _blocked_keys:
    pool = pool[
        (~pool["id"].astype(str).isin(_blocked_ids)) &
        (~pool.apply(_mk_key_from_row, axis=1).isin(_blocked_keys))
    ].copy()

def _eligible(df):
    # Exclude: already used (id/key), blocked (id/key), and enforce stock threshold
    ids = df["id"].astype(str)
    keys = df.apply(_mk_key_from_row, axis=1)
    return df[
        (~ids.isin(used_ids)) &
        (~keys.isin(used_keys)) &
        (~ids.isin(_blocked_ids)) &
        (~keys.isin(_blocked_keys)) &
        (df["stock"] >= min_stock)
    ]

def _pick_for_day(day, free_slots):
    if free_slots <= 0:
        return pd.DataFrame(columns=pool.columns)

    allowed_tiers = DAY_TIERS.get(day, [])
    need = free_slots

    # 1) strict: occasion + allowed tiers
    cand = _eligible(pool[
        (pool["occasion"].fillna("").str.lower() == DAY_OCCASION[day].lower()) &
        (pool["price_tier"].isin(allowed_tiers))
    ])
    pick = cand.sort_values(["segment_score","stock"], ascending=[False, False]).head(need)

    # 2) relax occasion, keep tiers
    if len(pick) < need:
        cand2 = _eligible(pool[(pool["price_tier"].isin(allowed_tiers))])
        extra = cand2.sort_values(["segment_score","stock"], ascending=[False, False]).head(need - len(pick))
        pick = pd.concat([pick, extra], ignore_index=True)

    # 3) seasonal fallback (last 12 months) if column exists
    if len(pick) < need and "OMT last offer date" in pool.columns:
        seasonal = pool.copy()
        seasonal["OMT last offer date"] = pd.to_datetime(seasonal["OMT last offer date"], errors="coerce")
        cut_start = datetime.today() - timedelta(days=365)
        cut_end = cut_start + timedelta(days=7)
        seasonal = seasonal[seasonal["OMT last offer date"].between(cut_start, cut_end)]
        cand3 = _eligible(seasonal)
        extra = cand3.sort_values(["segment_score","stock"], ascending=[False, False]).head(need - len(pick))
        pick = pd.concat([pick, extra], ignore_index=True)

    # 4) Top stock fallback
    if len(pick) < need:
        cand4 = _eligible(pool)
        extra = cand4.sort_values(["stock","segment_score"], ascending=[False, False]).head(need - len(pick))
        pick = pd.concat([pick, extra], ignore_index=True)

    return pick.head(free_slots)

# --- Day scope: allow single-day reshuffle if calendar_day is set (NEW) ---
_days_to_fill = [d for d in DAYS]
if _UF.get("calendar_day"):
    target = next((d for d in DAYS if d.lower() == str(_UF["calendar_day"]).lower()), None)
    if target:
        _days_to_fill = [target]
        print(f"🎯 Day-scoped reshuffle enabled for: {target}")

# Build calendar with locked first, then fill remaining slots (respect scope)
day_rows = []
for day in DAYS:
    slots = [None]*NUM_SLOTS

    # Place locked wines into their slots
    if not locked_df.empty:
        for _, r in locked_df[locked_df["day"] == day].iterrows():
            s = int(r["slot"])
            if 0 <= s < NUM_SLOTS:
                item_locked = {
                    "id": str(r.get("id") or "").strip(),
                    "wine": str(r.get("wine") or "").strip(),
                    "vintage": str(r.get("vintage") or "NV").strip(),
                    "locked": True
                }
                slots[s] = item_locked
                if item_locked["id"]:
                    used_ids.add(item_locked["id"])
                used_keys.add(_mk_key(item_locked))

    # Only auto-fill the requested day(s)
    if day in _days_to_fill:
        free_idx = [i for i in range(NUM_SLOTS) if slots[i] is None]
        picks = _pick_for_day(day, len(free_idx))

        # Fill free slots in order
        for i, (_, row) in zip(free_idx, picks.iterrows()):
            entry = {
                "id": str(row.get("id") or "").strip(),
                "wine": str(row.get("wine") or "").strip(),
                "vintage": str(row.get("vintage") or "NV").strip(),
                "full_type": row.get("full_type", "Unknown"),
                "region_group": row.get("region_group", "Unknown"),
                "stock": int(row.get("stock", 0)),
                "price_tier": row.get("price_tier", ""),
                "match_quality": "Auto",
                "locked": False
            }
            slots[i] = entry
            if entry["id"]:
                used_ids.add(entry["id"])
            used_keys.add(_mk_key(entry))

    # Materialize rows
    for s_idx, item in enumerate(slots):
        if item is None:
            continue
        out = {"day": day, "slot": s_idx, **item}
        day_rows.append(out)

weekly_calendar_df = pd.DataFrame(day_rows).sort_values(["day","slot"]).reset_index(drop=True)

# Ensure expected columns exist and types
for col in ["id","wine","vintage","full_type","region_group","stock","price_tier","locked"]:
    if col not in weekly_calendar_df.columns:
        weekly_calendar_df[col] = np.nan

weekly_calendar_df["stock"] = pd.to_numeric(weekly_calendar_df["stock"], errors="coerce").fillna(0).astype(int)
weekly_calendar_df["id"] = weekly_calendar_df["id"].astype(str).str.strip()
weekly_calendar_df.loc[weekly_calendar_df["id"].isin(["", "nan", "None"]), "id"] = None
weekly_calendar_df["price_tier"] = weekly_calendar_df["price_tier"].astype(str).replace({"nan": ""})

# --- Build day → list map once (no duplicates) ---
try:
    DAYS
except NameError:
    DAYS = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

weekly_calendar = {d: [] for d in DAYS}

def _safe_int(x, default=0):
    v = pd.to_numeric(x, errors="coerce")
    return int(v) if pd.notna(v) else default

for _, r in weekly_calendar_df.iterrows():
    d = (str(r.get("day") or "").strip() or "Monday")
    if d not in weekly_calendar:
        weekly_calendar[d] = []

    rid = r.get("id")
    vintage = r.get("vintage")
    full_type = r.get("full_type")
    region = r.get("region_group")
    tier = r.get("price_tier") or ""

    weekly_calendar[d].append({
        "id": "" if pd.isna(rid) else str(rid),
        "wine": r.get("wine") or r.get("name") or "Unknown",
        "vintage": vintage if pd.notna(vintage) else "NV",
        "full_type": full_type if (full_type is None or pd.notna(full_type)) else None,
        "region_group": region if (region is None or pd.notna(region)) else None,
        "stock": _safe_int(r.get("stock", 0)),
        "price_tier": tier,
        "locked": bool(r.get("locked", False)),
    })

print("📅 Calendar prepared (rows):", len(weekly_calendar_df))
print("🧩 Unique wines used:", weekly_calendar_df["id"].astype(str).nunique())
print("🔒 Locked kept:", int(weekly_calendar_df["locked"].sum()) if "locked" in weekly_calendar_df.columns else 0)
if _blocked_ids or _blocked_keys:
    print(f"⛔ Blocks respected: {len(_blocked_ids)} ids, {len(_blocked_keys)} keys")
if _UF.get("calendar_day"):
    print(f"🗓️ Filled day(s): {_days_to_fill}")


📅 Week selected: 34
🔍 Active UI filters (normalized):
{
  "loyalty": "all",
  "loyalty_levels": [],
  "wine_type": null,
  "bottle_size": null,
  "price_tier_bucket": "",
  "price_tiers": [],
  "last_stock": false,
  "last_stock_threshold": null,
  "seasonality_boost": false,
  "style": "default",
  "calendar_day": null,
  "blocked_ids": [],
  "blocked_keys": []
}
🔒 Locked slots in effect: 2
📅 Calendar prepared (rows): 35
🧩 Unique wines used: 34
🔒 Locked kept: 2


In [8]:
# --- CELL 6: Build schedule (style/day aware) and persist for API ---

import os
from time import perf_counter
from pathlib import Path
from datetime import datetime
import json
import pandas as pd
import numpy as np

# Blocks from UI (or params)
BLOCKED_IDS  = {str(x).strip() for x in (UI_FILTERS.get("blocked_ids")  or PARAMS.get("blocked_ids")  or []) if str(x).strip()}
BLOCKED_KEYS = {str(x).strip() for x in (UI_FILTERS.get("blocked_keys") or PARAMS.get("blocked_keys") or []) if str(x).strip()}

def _mk_key(it):
    return (str(it.get("id") or "").strip()
            or f"{(it.get('wine') or '').strip()}::{(it.get('vintage') or 'NV').strip()}")

t0 = perf_counter()

# ---------- Preconditions & fallbacks ----------
# Week number (prefer WEEK_NUMBER, then week_number)
try:
    WEEK_NUMBER = int(WEEK_NUMBER)
except Exception:
    try:
        WEEK_NUMBER = int(week_number)
    except Exception:
        WEEK_NUMBER = int(datetime.now().isocalendar().week)

# Calendar year (from Cell 1)
try:
    YEAR = int(CALENDAR_YEAR)
except Exception:
    YEAR = int(datetime.now().year)

assert 1 <= WEEK_NUMBER <= 53, f"Invalid WEEK_NUMBER={WEEK_NUMBER}"

# Paths and inputs
if 'OUTPUT_PATH' not in globals():
    raise NameError("OUTPUT_PATH is not defined (set in Cell 1).")
if 'UI_FILTERS' not in globals():
    UI_FILTERS = {}
if 'LOCKED_CALENDAR' not in globals():
    LOCKED_CALENDAR = {}

# Canonical calendar shape
NUM_SLOTS = globals().get("NUM_SLOTS", 5)
DAYS      = globals().get("DAYS", ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"])

# ---------- Helper fallbacks (only if missing) ----------
def _val(*candidates, default=None):
    for c in candidates:
        if c is not None and c != "":
            return c
    return default

if "_price_tier_of" not in globals():
    def _price_tier_of(it):
        return _val(
            it.get("price_tier_bucket"),
            it.get("price_bucket"),
            it.get("price_category"),
            it.get("price_tier"),
            it.get("priceTier"),
            it.get("tier"),
            default=""
        )

if "_stock_of" not in globals():
    def _stock_of(it):
        v = _val(it.get("stock"), it.get("stock_count"), it.get("qty"), it.get("quantity"))
        try:
            return int(v)
        except Exception:
            return None

if "_matches_filters" not in globals():
    # Minimal filter: last_stock + single price_tier_bucket + wine_type contains
    def _type_name_of(it):
        s = (it.get("full_type") or it.get("type") or "").lower()
        name = (it.get("wine") or it.get("name") or "").lower()
        return s, name

    def _matches_filters(it, f):
        # last stock
        if f.get("last_stock"):
            thr = int(f.get("last_stock_threshold") or 10)
            st = _stock_of(it)
            if st is None or not (st <= thr):   # <= for consistency with other cells
                return False

        # price tier
        ptb = (f.get("price_tier_bucket") or "").strip()
        if ptb and _price_tier_of(it) != ptb:
            return False

        # wine type (broad contains)
        wt = f.get("wine_type")
        if wt:
            want = str(wt).lower()
            s, name = _type_name_of(it)
            matched = (
                want in s
                or (want == "rosé" and ("rose" in s or "rosé" in s))
                or (want == "rose" and ("rosé" in s or "rose" in s))
                or (want == "red" and ("red" in s or "bordeaux" in name))
                or (want == "sparkling" and any(k in s for k in ["spark", "champ", "cava", "prosecco", "spumante"]))
            )
            if not matched:
                return False
        return True

# History map for cooldown (should be set in Cell 1; fallback to {})
HISTORY_MAP = globals().get("HISTORY_MAP", {}) or {}

# ---------- Style-aware scoring & constraints ----------
STYLE = (UI_FILTERS.get("style") or "default").strip().lower()
STYLE_CAT  = STYLE == "cat"
STYLE_NIGO = STYLE == "nigo"

DAY_TIER_PREF_CAT = {
    "Monday":   ["Mid-range","Premium"],
    "Tuesday":  ["Premium","Luxury"],
    "Wednesday":["Premium","Luxury"],
    "Thursday": ["Premium","Luxury","Ultra Luxury"],
    "Friday":   ["Luxury","Ultra Luxury"],
    "Saturday": ["Luxury","Ultra Luxury"],
    "Sunday":   ["Mid-range","Premium"]
}
DAY_TIER_PREF_NIGO = {
    "Monday":   ["Budget","Mid-range"],
    "Tuesday":  ["Mid-range","Premium"],
    "Wednesday":["Premium"],
    "Thursday": ["Premium"],
    "Friday":   ["Premium","Luxury"],
    "Saturday": ["Premium","Luxury"],
    "Sunday":   ["Budget","Mid-range","Premium"]
}
DAY_TIER_PREF = DAY_TIER_PREF_CAT if STYLE_CAT else (DAY_TIER_PREF_NIGO if STYLE_NIGO else {
    d:["Budget","Mid-range","Premium","Luxury","Ultra Luxury"] for d in DAYS
})

COOLDOWN_DAYS = 7 if STYLE_CAT else (35 if STYLE_NIGO else 21)
MAX_ULTRA     = 3 if STYLE_CAT else (1 if STYLE_NIGO else 2)

def _last_campaign_dt(it):
    id_  = str(it.get("id") or "").strip()
    key  = id_ or f"{(it.get('wine') or '').strip()}::{(it.get('vintage') or 'NV').strip()}"
    iso  = (HISTORY_MAP.get(key, {}) or {}).get("last_campaign_date") or it.get("last_campaign_date") or ""
    try:
        return datetime.fromisoformat(iso) if iso else None
    except Exception:
        return None

def _is_ultra(it):
    return (_price_tier_of(it) or "").strip() == "Ultra Luxury"

def _style_base_score(it):
    base = 0.0
    mq = str(it.get("match_quality") or "").lower()
    if "exact" in mq:  base += 3.0
    elif "high" in mq: base += 2.0
    elif "history" in mq: base += 0.8

    try:
        base += 0.5 * float(it.get("avg_cpi_score") or 0)
    except Exception:
        pass

    stock = _stock_of(it) or 0
    stock_norm = max(0.0, min(1.0, stock/150))
    base += (1.4 if STYLE_CAT else 0.3) * stock_norm

    tier = _price_tier_of(it)
    if STYLE_CAT:
        if tier in ("Luxury","Ultra Luxury"): base += 0.6
        elif tier == "Premium":               base += 0.4
        elif tier == "Mid-range":             base += 0.2
    elif STYLE_NIGO:
        if tier == "Premium":                 base += 0.5
        elif tier == "Luxury":                base += 0.2
        elif tier == "Ultra Luxury":          base -= 0.3
        elif tier == "Budget":                base += 0.1

    dt = _last_campaign_dt(it)
    if dt:
        age_days = max(0, (datetime.now() - dt).days)
        strength = 1.8 if STYLE_NIGO else (0.6 if STYLE_CAT else 1.0)
        penalty = max(0.0, strength * (1.0 - min(age_days, COOLDOWN_DAYS)/COOLDOWN_DAYS))
        base -= penalty

    return base

def _day_adjust_score(it, day, region_counts):
    s = 0.0
    if _price_tier_of(it) in DAY_TIER_PREF.get(day, []):
        s += 0.35 if STYLE_CAT else 0.45
    reg = str(it.get("region_group") or "").strip().lower()
    if STYLE_NIGO and reg:
        used = region_counts.get(reg, 0)
        if used >= 2:
            s -= 0.4 + 0.2*(used-2)
    return s

def _best_for_day(day, pool_list, used_keys, ultra_used, region_counts):
    best_idx, best_score, best_item = None, -1e9, None
    for i, it in enumerate(pool_list):
        if _is_ultra(it) and ultra_used >= MAX_ULTRA:
            continue
        s = _style_base_score(it) + _day_adjust_score(it, day, region_counts)
        if s > best_score:
            best_idx, best_score, best_item = i, s, it
    return best_idx, best_item

# ---------- Load base candidates (prefer year+week; then legacy; then slot-structured `weekly_calendar`) ----------
def _load_base_week():
    candidates = [
        OUTPUT_PATH / f"weekly_campaign_schedule_{YEAR}_week_{WEEK_NUMBER}.json",  # preferred
        OUTPUT_PATH / f"weekly_campaign_schedule_week_{WEEK_NUMBER}.json",         # legacy
    ]
    for p in candidates:
        if p.exists():
            try:
                return json.loads(p.read_text(encoding="utf-8"))
            except Exception:
                pass
    # fallback: slot-structured weekly_calendar from Cell 5
    if 'weekly_calendar' in globals() and isinstance(weekly_calendar, dict):
        return {d: [x for x in (weekly_calendar.get(d) or []) if x] for d in DAYS}
    return {d: [] for d in DAYS}

base_week = _load_base_week()

# ---------- Start with UI-locked slots ----------
out = {d: [None]*NUM_SLOTS for d in DAYS}
used_keys = set()
ultra_used = 0
region_counts = {}

for day in DAYS:
    slots = (LOCKED_CALENDAR or {}).get(day) or []
    for idx, it in enumerate(slots[:NUM_SLOTS]):
        if not it:
            continue
        item = {
            "id": it.get("id") or "",
            "wine": it.get("wine") or it.get("name") or "Unknown",
            "vintage": it.get("vintage") or "NV",
            "full_type": it.get("full_type") or it.get("type") or "",
            "region_group": it.get("region_group") or "",
            "price_tier": _price_tier_of(it) or "",
            "stock": _stock_of(it),
            "match_quality": it.get("match_quality") or "Locked",
            "avg_cpi_score": it.get("avg_cpi_score") or 0,
            "locked": True,
            "last_campaign_date": it.get("last_campaign_date") or "",
        }
        out[day][idx] = item
        k = _mk_key(item)
        used_keys.add(k)
        if _is_ultra(item): ultra_used += 1
        reg = str(item.get("region_group") or "").strip().lower()
        if reg: region_counts[reg] = region_counts.get(reg, 0) + 1

# ---------- Build filtered pool (dedup against locks, honor blocks & UI filters) ----------
pool = []
for d in DAYS:
    for it in base_week.get(d, []):
        if not _matches_filters(it, UI_FILTERS):
            continue
        kid = str(it.get("id") or "").strip()
        key = _mk_key(it)
        if (kid and kid in BLOCKED_IDS) or (key and key in BLOCKED_KEYS):
            continue
        if key in used_keys:
            continue
        pool.append(it)

# ---------- Fill remaining slots day-by-day ----------
for day in DAYS:
    for slot in range(NUM_SLOTS):
        if out[day][slot] is not None:
            continue
        idx, cand = _best_for_day(day, pool, used_keys, ultra_used, region_counts)
        if idx is None or cand is None:
            break
        pool.pop(idx)
        item = {
            "id": cand.get("id") or "",
            "wine": cand.get("wine") or cand.get("name") or "Unknown",
            "vintage": cand.get("vintage") or "NV",
            "full_type": cand.get("full_type") or cand.get("type") or "",
            "region_group": cand.get("region_group") or "",
            "price_tier": _price_tier_of(cand),
            "stock": _stock_of(cand),
            "match_quality": cand.get("match_quality") or "Auto",
            "avg_cpi_score": cand.get("avg_cpi_score") or 0,
            "locked": bool(cand.get("locked", False)),
            "last_campaign_date": cand.get("last_campaign_date") or "",
        }
        out[day][slot] = item
        k = _mk_key(item)
        used_keys.add(k)
        if _is_ultra(item): ultra_used += 1
        reg = str(item.get("region_group") or "").strip().lower()
        if reg: region_counts[reg] = region_counts.get(reg, 0) + 1

# ---------- Persist flat day arrays for API (/api/schedule) ----------
out_flat = {d: [x for x in out[d] if x is not None] for d in DAYS}

def atomic_write_text(path: Path, text: str):
    tmp = path.with_suffix(path.suffix + ".tmp")
    tmp.write_text(text, encoding="utf-8")
    os.replace(tmp, path)

year_json   = OUTPUT_PATH / f"weekly_campaign_schedule_{YEAR}_week_{WEEK_NUMBER}.json"
legacy_json = OUTPUT_PATH / f"weekly_campaign_schedule_week_{WEEK_NUMBER}.json"
atomic_write_text(year_json,   json.dumps(out_flat, indent=2))
atomic_write_text(legacy_json, json.dumps(out_flat, indent=2))

# index
index_path = OUTPUT_PATH / "schedule_index.json"
try:
    idx = json.loads(index_path.read_text(encoding="utf-8")) if index_path.exists() else {}
except Exception:
    idx = {}
idx.setdefault(str(YEAR), {})[str(WEEK_NUMBER)] = {
    "json": year_json.name,
    "updated_at": datetime.now().isoformat(timespec="seconds")
}
idx["_latest_year"] = str(YEAR)
idx["_latest_week"] = str(WEEK_NUMBER)
atomic_write_text(index_path, json.dumps(idx, indent=2))

print(f"✅ Rebuilt {YEAR}-W{WEEK_NUMBER} ({STYLE or 'default'}) → {year_json.name}")
print(f"⏱️ Selection duration: {round(perf_counter() - t0, 2)}s")
print(f"⛔ Blocks: {len(BLOCKED_IDS)} ids, {len(BLOCKED_KEYS)} keys | 🔒 Locks kept.")


✅ Rebuilt 2025-W34 (default) → weekly_campaign_schedule_2025_week_34.json
⏱️ Selection duration: 0.05s
⛔ Blocks: 0 ids, 0 keys | 🔒 Locks kept.


In [10]:
# --- CELL 7: Fill & persist (strict-pool; honors last_stock, locks & blocks) ---

import os
import json
from datetime import datetime
from pathlib import Path

assert WEEK_NUMBER, "WEEK_NUMBER missing"
STYLE = (UI_FILTERS.get("style") or "default").strip().lower()

NUM_SLOTS = 5
DAYS = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

# Year (aware calendar)
try:
    YEAR = int(CALENDAR_YEAR)
except Exception:
    YEAR = int(datetime.now().year)

# Blocks coming from UI/params
BLOCKED_IDS  = {str(x).strip() for x in (UI_FILTERS.get("blocked_ids")  or PARAMS.get("blocked_ids")  or []) if str(x).strip()}
BLOCKED_KEYS = {str(x).strip() for x in (UI_FILTERS.get("blocked_keys") or PARAMS.get("blocked_keys") or []) if str(x).strip()}

def _mk_key(it):
    i = str(it.get("id") or "").strip()
    return i or f"{(it.get('wine') or '').strip()}::{(it.get('vintage') or 'NV').strip()}"

def _tier(it):
    return (it.get("price_tier") or it.get("price_tier_bucket") or "").strip()

def _stock(it):
    try:
        return int(float(it.get("stock", 0)))
    except Exception:
        return 0

def _type_str(it):
    return (it.get("full_type") or it.get("type") or "").lower()

# recency helper (self-contained for this cell)
HISTORY_MAP = globals().get("HISTORY_MAP", {}) or {}
def _last_campaign_dt(it):
    id_  = str(it.get("id") or "").strip()
    key  = id_ or f"{(it.get('wine') or '').strip()}::{(it.get('vintage') or 'NV').strip()}"
    iso  = (HISTORY_MAP.get(key, {}) or {}).get("last_campaign_date") or it.get("last_campaign_date") or ""
    try:
        return datetime.fromisoformat(iso) if iso else None
    except Exception:
        return None

def _match_filters(it, f):
    # price tiers
    pts = set((f.get("price_tiers") or []))
    ptb = (f.get("price_tier_bucket") or "").strip()
    if ptb:
        pts.add(ptb)
    if pts and _tier(it) not in pts:
        return False
    # wine type
    wt = (f.get("wine_type") or "").strip().lower()
    if wt and wt not in _type_str(it):
        return False
    # last stock or minimum availability
    if f.get("last_stock"):
        thr = int(f.get("last_stock_threshold") or 10)
        if not (0 < _stock(it) <= thr):        # ≤ for consistency with other cells
            return False
    else:
        if _stock(it) < 6:
            return False
    return True

def _day_pref(day):
    if STYLE == "cat":
        return {
            "Monday":["Mid-range","Premium"], "Tuesday":["Premium","Luxury"],
            "Wednesday":["Premium","Luxury"], "Thursday":["Premium","Luxury","Ultra Luxury"],
            "Friday":["Luxury","Ultra Luxury"], "Saturday":["Luxury","Ultra Luxury"],
            "Sunday":["Mid-range","Premium"]
        }.get(day, [])
    if STYLE == "nigo":
        return {
            "Monday":["Budget","Mid-range"], "Tuesday":["Mid-range","Premium"],
            "Wednesday":["Premium"], "Thursday":["Premium"],
            "Friday":["Premium","Luxury"], "Saturday":["Premium","Luxury"],
            "Sunday":["Budget","Mid-range","Premium"]
        }.get(day, [])
    return ["Budget","Mid-range","Premium","Luxury","Ultra Luxury"]

def _score(it, day):
    base = float(it.get("avg_cpi_score") or 0) * 0.5
    if _tier(it) in _day_pref(day):
        base += 0.4
    # recency cooldown
    dt = _last_campaign_dt(it)
    if dt:
        age = max(0, (datetime.now() - dt).days)
        cool = 7 if STYLE == "cat" else (35 if STYLE == "nigo" else 21)
        penalty = max(0.0, (1.2 if STYLE == "nigo" else 0.6) * (1.0 - min(age, cool)/cool))
        base -= penalty
    # stock pressure
    s = _stock(it)
    base += (1.2 if STYLE == "cat" else 0.3) * min(1.0, s/150.0)
    return base

def _load_base_week():
    # Prefer year+week; fallback to legacy; fallback to slot-structured `weekly_calendar`
    candidates = [
        OUTPUT_PATH / f"weekly_campaign_schedule_{YEAR}_week_{WEEK_NUMBER}.json",
        OUTPUT_PATH / f"weekly_campaign_schedule_week_{WEEK_NUMBER}.json",
    ]
    for p in candidates:
        if p.exists():
            try:
                return json.loads(p.read_text(encoding="utf-8"))
            except Exception:
                pass
    if 'weekly_calendar' in globals() and isinstance(weekly_calendar, dict):
        return {d: [x for x in (weekly_calendar.get(d) or []) if x] for d in DAYS}
    return {d: [] for d in DAYS}

# 1) Load flat base week so we respect existing calendar content
base_week = _load_base_week()

# 2) Build a strict candidate POOL from base_week that also satisfies UI filters + blocks
POOL = []
for day in DAYS:
    for it in base_week.get(day, []):
        if not _match_filters(it, UI_FILTERS):
            continue
        kid = str(it.get("id") or "").strip()
        key = _mk_key(it)
        if (kid and kid in BLOCKED_IDS) or (key and key in BLOCKED_KEYS):
            continue
        POOL.append(it)

# 3) Seed with locked slots (persisted + UI overlay done in Cell 1 as EFFECTIVE_LOCKS/LOCKED_CALENDAR)
locked = globals().get("LOCKED_CALENDAR") or {}
out = {d: [None]*NUM_SLOTS for d in DAYS}
used = set()

for day in DAYS:
    slots = locked.get(day) or []
    for idx, it in enumerate(slots[:NUM_SLOTS]):
        if not it:
            continue
        item = {
            "id": it.get("id") or "",
            "wine": it.get("wine") or it.get("name") or "Unknown",
            "vintage": it.get("vintage") or "NV",
            "full_type": it.get("full_type") or it.get("type") or "",
            "region_group": it.get("region_group") or "",
            "price_tier": _tier(it),
            "stock": _stock(it),
            "match_quality": it.get("match_quality") or "Locked",
            "avg_cpi_score": it.get("avg_cpi_score") or 0,
            "locked": True,
            "last_campaign_date": it.get("last_campaign_date") or "",
        }
        out[day][idx] = item
        used.add(_mk_key(item))

# 4) Per-day pick (strictly from POOL)
for day in DAYS:
    for s in range(NUM_SLOTS):
        if out[day][s] is not None:
            continue
        best_i, best_sc = None, -1e9
        for i, it in enumerate(POOL):
            k = _mk_key(it)
            if k in used:
                continue
            sc = _score(it, day)
            if sc > best_sc:
                best_i, best_sc = i, sc
        if best_i is None:
            break
        it = POOL.pop(best_i)
        item = {
            "id": it.get("id") or "",
            "wine": it.get("wine") or it.get("name") or "Unknown",
            "vintage": it.get("vintage") or "NV",
            "full_type": it.get("full_type") or it.get("type") or "",
            "region_group": it.get("region_group") or "",
            "price_tier": _tier(it),
            "stock": _stock(it),
            "match_quality": it.get("match_quality") or "Auto",
            "avg_cpi_score": it.get("avg_cpi_score") or 0,
            "locked": bool(it.get("locked", False)),
            "last_campaign_date": it.get("last_campaign_date") or "",
        }
        out[day][s] = item
        used.add(_mk_key(item))

# 5) Persist for API (year+week primary; legacy mirror). Atomic writes + index.
def atomic_write_text(path: Path, text: str):
    tmp = path.with_suffix(path.suffix + ".tmp")
    tmp.write_text(text, encoding="utf-8")
    os.replace(tmp, path)

out_flat = {d: [x for x in out[d] if x is not None] for d in DAYS}

year_json   = OUTPUT_PATH / f"weekly_campaign_schedule_{YEAR}_week_{WEEK_NUMBER}.json"
legacy_json = OUTPUT_PATH / f"weekly_campaign_schedule_week_{WEEK_NUMBER}.json"
atomic_write_text(year_json,   json.dumps(out_flat, indent=2))
atomic_write_text(legacy_json, json.dumps(out_flat, indent=2))

# Update index
index_path = OUTPUT_PATH / "schedule_index.json"
try:
    idx = json.loads(index_path.read_text(encoding="utf-8")) if index_path.exists() else {}
except Exception:
    idx = {}
idx.setdefault(str(YEAR), {})[str(WEEK_NUMBER)] = {
    "json": year_json.name,
    "updated_at": datetime.now().isoformat(timespec="seconds")
}
idx["_latest_year"] = str(YEAR)
idx["_latest_week"] = str(WEEK_NUMBER)
atomic_write_text(index_path, json.dumps(idx, indent=2))

print(f"✅ Week {YEAR}-W{WEEK_NUMBER} rebuilt ({STYLE}) → {year_json.name}")
print(f"⛔ Blocks honored: {len(BLOCKED_IDS)} ids, {len(BLOCKED_KEYS)} keys | 🔒 Locks kept: {sum(1 for d in DAYS for x in (globals().get('LOCKED_CALENDAR', {}).get(d) or []) if x)}")


✅ Week 2025-W34 rebuilt (default) → weekly_campaign_schedule_2025_week_34.json
⛔ Blocks honored: 0 ids, 0 keys | 🔒 Locks kept: 2


In [ ]:
# --- CELL 8: Build UI convenience payloads and save mirrors (fixed & order-safe) ---

from pathlib import Path
from statistics import mean
import json
import pandas as pd
import numpy as np
from datetime import datetime

YEAR, WEEK = int(CALENDAR_YEAR), int(WEEK_NUMBER)

weekly_json_path = OUTPUT_PATH / f"weekly_campaign_schedule_{YEAR}_week_{WEEK}.json"
# Fallback: legacy
if not weekly_json_path.exists():
    weekly_json_path = OUTPUT_PATH / f"weekly_campaign_schedule_week_{WEEK}.json"

# ... build `flat` (same as you have) ...

# UI slot calendar (7 x NUM_SLOTS)
weekly_calendar_slots = {d: (flat[d] + [None]*NUM_SLOTS)[:NUM_SLOTS] for d in DAYS}

ui_output_data = {
    "top_recommendations": top_recs,
    "weekly_calendar": weekly_calendar_slots,
    "cpi_score": cpi_score,
    "client_prefs": []
}

# Persist API (day arrays)
api_weekly_out = OUTPUT_PATH / f"weekly_campaign_schedule_{YEAR}_week_{WEEK}.json"
atomic_write_text(api_weekly_out, json.dumps(flat, indent=2))

# UI mirrors
IRON_DATA_PATH.mkdir(parents=True, exist_ok=True)
ui_json   = IRON_DATA_PATH / f"weekly_campaign_schedule_{YEAR}_week_{WEEK}.json"
ui_pkl    = IRON_DATA_PATH / f"weekly_campaign_schedule_{YEAR}_week_{WEEK}.pkl"
ui_uijson = IRON_DATA_PATH / f"weekly_campaign_schedule_{YEAR}_week_{WEEK}.ui.json"  # <- new preferred

with open(ui_json, "w", encoding="utf-8") as f:
    json.dump(weekly_calendar_slots, f, indent=4, default=str)
pd.to_pickle(weekly_calendar_slots, ui_pkl)

with open(ui_uijson, "w", encoding="utf-8") as f:
    json.dump({"weekly_calendar": weekly_calendar_slots}, f, indent=2, default=str)

# Legacy mirrors
with open(IRON_DATA_PATH / f"weekly_campaign_schedule_week_{WEEK}.json", "w", encoding="utf-8") as f:
    json.dump(weekly_calendar_slots, f, indent=4, default=str)
pd.to_pickle(weekly_calendar_slots, IRON_DATA_PATH / f"weekly_campaign_schedule_week_{WEEK}.pkl")

print(f"✅ Saved UI mirrors for {YEAR}-W{WEEK}")



# --- Guards / defaults ---
try:
    WEEK_NUMBER = int(WEEK_NUMBER)
except Exception:
    WEEK_NUMBER = int(datetime.now().isocalendar().week)

if 'OUTPUT_PATH' not in globals():
    OUTPUT_PATH = (Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" /
                   "Puzzle_control_Reports" / "IRON_DATA" / "_output")
if 'IRON_DATA_PATH' not in globals():
    IRON_DATA_PATH = OUTPUT_PATH.parent

OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
IRON_DATA_PATH.mkdir(parents=True, exist_ok=True)

NUM_SLOTS = int(globals().get("NUM_SLOTS", 5))
DAYS      = list(globals().get("DAYS", ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]))

weekly_json_path = OUTPUT_PATH / f"weekly_campaign_schedule_week_{WEEK_NUMBER}.json"

# --- Source of truth for "flat" (day -> list of items) ---
flat = None

# 1) Preferred: what Cell 6B just produced
if 'out_flat' in globals() and isinstance(out_flat, dict):
    flat = out_flat

# 2) If we only have a slot-structured weekly_calendar, convert it
if flat is None and 'weekly_calendar' in globals() and isinstance(weekly_calendar, dict):
    flat = {d: [x for x in (weekly_calendar.get(d) or []) if x] for d in DAYS}

# 3) Otherwise load from disk (written by Ignition or 6B earlier)
if flat is None and weekly_json_path.exists():
    try:
        flat = json.loads(weekly_json_path.read_text(encoding="utf-8"))
    except Exception:
        flat = None

# 4) Final fallback
if flat is None:
    flat = {d: [] for d in DAYS}

# Normalize: lists only, drop None, clamp to NUM_SLOTS
flat = {d: [x for x in (flat.get(d) or []) if x is not None][:NUM_SLOTS] for d in DAYS}

# --- Save a debug copy without None (now that `flat` exists) ---
flat_no_none_path = OUTPUT_PATH / f"weekly_campaign_schedule_flat_week_{WEEK_NUMBER}.json"
flat_no_none_path.write_text(json.dumps(flat, indent=2), encoding="utf-8")

# --- Build slot-structured calendar (7 × NUM_SLOTS) for UI mirrors ---
weekly_calendar_slots = {d: (flat[d] + [None]*NUM_SLOTS)[:NUM_SLOTS] for d in DAYS}

# --- Simple CPI preview & top rec preview ---
def _to_float(x):
    try:
        v = float(x)
        return v if np.isfinite(v) else 0.0
    except Exception:
        return 0.0

all_items = [it for d in DAYS for it in flat.get(d, [])]
cpi_vals = [_to_float(it.get("avg_cpi_score")) for it in all_items if "avg_cpi_score" in it]
cpi_score = round(mean(cpi_vals), 2) if cpi_vals else 0.0

top_recs = []
first_day = next((d for d in DAYS if flat.get(d)), None)
if first_day and flat[first_day]:
    f = flat[first_day][0]
    top_recs.append({"name": f.get("wine", "Unknown"), "vintage": f.get("vintage", "NV")})

ui_output_data = {
    "top_recommendations": top_recs,
    "weekly_calendar": weekly_calendar_slots,
    "cpi_score": cpi_score,
    "client_prefs": []
}

# --- Persist ---
# A) API file (day arrays) → used by /api/schedule
api_weekly_out = OUTPUT_PATH / f"weekly_campaign_schedule_week_{WEEK_NUMBER}.json"
api_weekly_out.write_text(json.dumps(flat, indent=2), encoding="utf-8")

# B) UI mirrors (slot arrays) → convenience copies
weekly_json = IRON_DATA_PATH / f"weekly_campaign_schedule_week_{WEEK_NUMBER}.json"
weekly_pkl  = IRON_DATA_PATH / f"weekly_campaign_schedule_week_{WEEK_NUMBER}.pkl"
fallback_json = IRON_DATA_PATH / "weekly_campaign_schedule.json"
fallback_pkl  = IRON_DATA_PATH / "weekly_campaign_schedule.pkl"

with open(weekly_json, "w", encoding="utf-8") as f:
    json.dump(ui_output_data["weekly_calendar"], f, indent=4, default=str)
pd.to_pickle(ui_output_data["weekly_calendar"], weekly_pkl)

with open(fallback_json, "w", encoding="utf-8") as f:
    json.dump(ui_output_data["weekly_calendar"], f, indent=4, default=str)
pd.to_pickle(ui_output_data["weekly_calendar"], fallback_pkl)

# --- Logs ---
print(f"✅ Saved UI mirrors for Week {WEEK_NUMBER}")
print(f"🗃️ API file: {api_weekly_out.name}")
print(f"🗃️ UI files: {weekly_json.name}, {weekly_pkl.name}")
print("UI_DATA_START_JSON_OUTPUT")
print("✅ UI_DATA_PREVIEW:", json.dumps({
    "cpi_score": ui_output_data["cpi_score"],
    "top_recommendations": ui_output_data["top_recommendations"]
}, indent=2))
print("UI_DATA_END_JSON_OUTPUT")
print("📅 Calendar Days:", list(ui_output_data["weekly_calendar"].keys()))
